# Parallelized version

In [1]:
import math
import random
import numpy as np
import time
from functools import reduce

## Initializing spark

In [2]:
from pyspark import SparkConf, SparkContext

In [3]:
conf = SparkConf().setAppName("appName").setMaster("local[*]")
sc = SparkContext(conf=conf)

22/12/06 14:39:50 WARN Utils: Your hostname, Alexs-MacBook-Air.local resolves to a loopback address: 127.0.0.1; using 192.168.1.134 instead (on interface en0)
22/12/06 14:39:50 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/12/06 14:39:51 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


## Functions

### Auxiliar functions

In [4]:
def str_to_number_list(line):
    numbers = line.replace(" ","").split(",")
    numbers = [float(number) for number in numbers]
    
    # Separate X and y
    numbers = (numbers[1:], numbers[0])
    
    return numbers

In [5]:
def trainTestSplit(dataset):
    train = dataset.filter(lambda x: x[2]==1)
    train = train.map(lambda x: (x[0], x[1]))
    test = dataset.filter(lambda x: x[2]==0)
    test = test.map(lambda x: (x[0], x[1]))
    return (train, test)

In [6]:
def parallelAssign2cluster_slower(x):
    global c
    min_distance = np.Inf
    centroid = 0
    for it, cen in enumerate(c.value):
        d = 0
        for y,z in zip(x[0],cen):
            d += abs(y-z)
        if d <= min_distance:
            min_distance = d
            centroid = it
        
    return (centroid, x[0])

In [7]:
def parallelKMeans_slower(data, K, n_iter):
    
    data_len = sc.broadcast(data.count())
    
    global c
    c = data.takeSample(0, K)
    c = [datum[0] for datum in c]
    c = sc.broadcast(c)
    
    for iter in range(n_iter):
        data_asigned = data.map(parallelAssign2cluster_slower)
        c = sc.broadcast(data_asigned.reduceByKey(lambda x,y: [q+w for q,w in zip(x,y)]) \
            .map(lambda x: [a/data_len.value for a in x[1]]).collect())
    
    return c.value

### Mandatory functions

In [8]:
def parallelReadFile(filename):
    dataset = sc.textFile(filename)
    
    # Extract the header from the data
    header = dataset.first()
    dataset = dataset.filter(lambda row: row != header)
    
    dataset = dataset.map(str_to_number_list)
    return dataset.sample(False, 1)

In [9]:
def parallelAssign2cluster(x):
    global c
    min_distance = np.Inf
    centroid = 0
    for it, cen in enumerate(c.value):
        d = reduce(lambda x,y:abs(x)+abs(y), [y-z for y,z in zip(x[0],cen)])
        if d <= min_distance:
            min_distance = d
            centroid = it
        
    return (centroid, x[0])

In [10]:
def parallelKMeans(data, K, n_iter):
    
    data_len = sc.broadcast(data.count())
    
    global c
    c = data.takeSample(0, K)
    c = [datum[0] for datum in c]
    c = sc.broadcast(c)
    
    for iter in range(n_iter):
        data_asigned = data.map(parallelAssign2cluster)
        c = sc.broadcast(data_asigned.reduceByKey(lambda x,y: [q+w for q,w in zip(x,y)]) \
            .map(lambda x: [a/data_len.value for a in x[1]]).collect())
    
    return c.value

## Testing

In [12]:
RDD_Xy = parallelReadFile("../data/tot_mnist_shuf.csv")
RDD_Xy.count()

70000

In [51]:
centroids = parallelKMeans(RDD_Xy, 3, 2)

In [12]:
elapsed_times = []

core_list = list(range(1,9,1))
cluster_list = [3,5,7,8,9,10,11]
functions = [parallelKMeans, parallelKMeans_slower]

for cores in core_list:
    for clusters in cluster_list:
        for Kmeans in functions:
            sc.stop()
            conf = SparkConf().setAppName("appName").setMaster(f"local[{cores}]")
            sc = SparkContext(conf=conf)

            print(f"---------- Starting execution with {cores} cores and {clusters} clusters ----------")

            start = time.time()

            # ---------- Execution ----------

            RDD_Xy = parallelReadFile("../data/tot_mnist_shuf.csv")
            Kmeans(RDD_Xy, clusters, 5)

            # ---------- Execution ----------

            end = time.time()
            print(f"---------- Finished execution with {cores} cores and {clusters} clusters ----------")
            elapsed_time = end - start
            elapsed_times.append(
                {
                    "Number of Cores": cores,
                    "Number of Clusters": clusters,
                    "Function used": Kmeans,
                    "elapsed_time": elapsed_time
                }
            )
            print(f"Elapsed time for cores {cores} is {elapsed_time} seconds")

---------- Starting execution with 1 cores and 3 clusters ----------


---------- Finished execution with 1 cores and 3 clusters ----------
Elapsed time for cores 1 is 279.2463307380676 seconds
---------- Starting execution with 1 cores and 3 clusters ----------


---------- Finished execution with 1 cores and 3 clusters ----------
Elapsed time for cores 1 is 175.4063618183136 seconds
---------- Starting execution with 1 cores and 5 clusters ----------


---------- Finished execution with 1 cores and 5 clusters ----------
Elapsed time for cores 1 is 391.22597789764404 seconds
---------- Starting execution with 1 cores and 5 clusters ----------


---------- Finished execution with 1 cores and 5 clusters ----------
Elapsed time for cores 1 is 217.99994802474976 seconds
---------- Starting execution with 1 cores and 7 clusters ----------


...

---------- Finished execution with 8 cores and 9 clusters ----------
Elapsed time for cores 8 is 136.7139129638672 seconds
---------- Starting execution with 8 cores and 10 clusters ----------


---------- Finished execution with 8 cores and 10 clusters ----------
Elapsed time for cores 8 is 272.82360887527466 seconds
---------- Starting execution with 8 cores and 10 clusters ----------


---------- Finished execution with 8 cores and 10 clusters ----------
Elapsed time for cores 8 is 168.7591049671173 seconds
---------- Starting execution with 8 cores and 11 clusters ----------


---------- Finished execution with 8 cores and 11 clusters ----------
Elapsed time for cores 8 is 326.1905369758606 seconds
---------- Starting execution with 8 cores and 11 clusters ----------


---------- Finished execution with 8 cores and 11 clusters ----------
Elapsed time for cores 8 is 177.2009117603302 seconds


In [ ]:
elapsed_times

[{'Number of Cores': 1,
  'Number of Clusters': 3,
  'Function used': <function __main__.parallelKMeans(data, K, n_iter)>,
  'elapsed_time': 279.2463307380676},
 {'Number of Cores': 1,
  'Number of Clusters': 3,
  'Function used': <function __main__.parallelKMeans_slower(data, K, n_iter)>,
  'elapsed_time': 175.4063618183136},
 {'Number of Cores': 1,
  'Number of Clusters': 5,
  'Function used': <function __main__.parallelKMeans(data, K, n_iter)>,
  'elapsed_time': 391.22597789764404},
 {'Number of Cores': 1,
  'Number of Clusters': 5,
  'Function used': <function __main__.parallelKMeans_slower(data, K, n_iter)>,
  'elapsed_time': 217.99994802474976},
 {'Number of Cores': 1,
  'Number of Clusters': 7,
  'Function used': <function __main__.parallelKMeans(data, K, n_iter)>,
  'elapsed_time': 2198.4438829421997},
 {'Number of Cores': 1,
  'Number of Clusters': 7,
  'Function used': <function __main__.parallelKMeans_slower(data, K, n_iter)>,
  'elapsed_time': 302.5171310901642},
 {'Number 